In [1]:
import psycopg2
import datetime
import getpass
import pandas as pd
import datetime
import pyodbc
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
teamproductyear_id = 123 #1,...,123 no 85

In [3]:
# connect to SQL Server.
server = '34.206.73.189' 
database = 'datascience' 
username = getpass.getpass(prompt='Enter your username') 
password = getpass.getpass(prompt='Enter your password')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = f'''select teamproductyearid,lkupclientid,clientcode,productgrouping,trainseasonyear,testseasonyear,facttestyear,facttestprevyear,stlrDBName from ds.productyear_all r where teamproductyearid ={teamproductyear_id} ;'''
dfparam3 = pd.read_sql(query, cnxn)
cnxn.commit()
cursor.close()

In [4]:
dfparam3.head()

,teamproductyearid,lkupclientid,clientcode,productgrouping,trainseasonyear,testseasonyear,facttestyear,facttestprevyear,stlrDBName
0,123,55,coyotes,Partial Season,2019,2021,2021,None,stlrCoyotes


In [5]:
client_id = dfparam3._get_value(0,'lkupclientid')
client_code= dfparam3._get_value(0,'clientcode')
product_grouping =dfparam3._get_value(0,'productgrouping') 
stlrdbname= dfparam3._get_value(0,'stlrDBName') 

In [6]:
# connect to SQL Server.
database = 'datascience'
cnxn5 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor5 = cnxn5.cursor()
MAXINSERTDATE =  f'''select MAX(cast(insertDate as date)) as maxInsertDate from ds.customerScores where lkupclientid ={client_id}  and productGrouping in({"'"+ str(product_grouping) + "'"}) ;'''
dfMaxDate1 = pd.read_sql(MAXINSERTDATE, cnxn5)
cnxn5.commit()
cursor5.close()
dfMaxDate1

,maxInsertDate
0,2021-11-16


In [9]:
insert_date= dfMaxDate1['maxInsertDate']
max_insert_date=insert_date[0]
str_insert_date=str(max_insert_date)
str_insert_date

'2021-11-16'

In [8]:
today = datetime.datetime.now()
today_date = today.strftime("%Y-%m-%d")
today_date

'2021-11-17'

In [10]:
if str_insert_date == today_date:
    print(client_code+' '+product_grouping+' Has'+' '+ str(cnt)+' '+'orphaned dimCustomerMasterId records in factCustomerRetentionScore with dimCustomerMaster table')
    print('Is Updated')
elif str_insert_date != today_date:
    print('Is Not Updated')

Is Not Updated


### select the MAX(dimDateId) for the specific lkupclientId and Product in factCustomerRetentionScore

In [9]:
# connect to SQL Server.
database = stlrdbname
cnxn2 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor2 = cnxn2.cursor()
MAXDIMDATE =  f'''select MAX(dimDateId) as maxDimDateId from dw.factCustomerretentionScore where lkupclientid ={client_id}  and product in({"'"+ str(product_grouping) + "'"}) and currVersnFlag =1;'''
dfMaxDate = pd.read_sql(MAXDIMDATE, cnxn2)
cnxn2.commit()
cursor2.close()
dfMaxDate

,maxDimDateId
0,20211116


### select dimCustomerMasterIds that exist in fatCustomerRetentionScore and don't exist in dimCustomerMaster table

In [10]:
# connect to SQL Server.
database = stlrdbname
cnxn3 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor3 = cnxn3.cursor()
dimCustomerMasterIDNull =  f'''SELECT Count(*) as cntDimCustomerMasterId FROM dw.factCustomerretentionScore t LEFT OUTER JOIN dw.dimCustomerMaster m ON m.dimCustomerMasterId= t.customerNumber where t.lkupclientid ={client_id}  AND t.product in({"'"+ str(product_grouping) + "'"}) AND currVersnFlag =1 AND m.dimCustomerMasterId IS NULL;'''
dfCNT = pd.read_sql(dimCustomerMasterIDNull, cnxn3)
cnxn3.commit()
cursor3.close()
dfCNT


,cntDimCustomerMasterId
0,0


In [11]:
cnt2= dfCNT['cntDimCustomerMasterId']
cnt=cnt2[0]
if cnt== 0:
    print('No Orphaned Records')
else:  
    print(client_code+' '+product_grouping+' Has'+' '+ str(cnt)+' '+'orphaned dimCustomerMasterId records in factCustomerRetentionScore with dimCustomerMaster table')

No Orphaned Records


### select The number of records group by each year for one specific lkupClientId and Product

In [12]:
# connect to SQL Server.
database = stlrdbname
cnxn4 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor4 = cnxn4.cursor()
cntPerYear =  f'''select seasonYear, count(*) cntPerYear from dw.factCustomerRetentionScore where lkupclientid ={client_id}  AND product in({"'"+ str(product_grouping) + "'"}) AND currVersnFlag =1 group by seasonYear;'''
dfcntPerYear = pd.read_sql(cntPerYear, cnxn4)
cnxn4.commit()
cursor4.close()
dfcntPerYear

,seasonYear,cntPerYear
0,2021,333


### select number of records which the testSeasonYear in factCustomerRetentionScore table is not equal to facttestyear in ds.productyear_all table where currVersnFlag = 1 and they don't have data for previous year

In [13]:
# connect to SQL Server.
database = stlrdbname
cnxn5 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor4 = cnxn5.cursor()
testSeasonYear =  f'''select count(*)  from (select lkupClientId,product,seasonYear testSeasonYear from dw.factCustomerRetentionScore t where t.lkupclientid ={client_id}  AND product in({"'"+ str(product_grouping) + "'"}) AND currVersnFlag = 1 ) b INNER JOIN [DataScience].[DS].[productyear_all] p on b.lkupClientId=p.lkupclientid AND b.product= p.productgrouping where b.testSeasonYear <> p.facttestyear and p.isprevyearflag = 0;'''
dfTestSeasonYear = pd.read_sql(testSeasonYear, cnxn5)
cnxn5.commit()
cursor4.close()
dfTestSeasonYear

,
0,0
